# Importacion de librerias y conexión

In [43]:
import mysql.connector
import pandas as pd
import numpy as np
import regex as re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [45]:
conn = mysql.connector.connect(
    host='212.227.90.6',
    user='Equipo19',
    password='E1q2u3i4p5o19',
    database='Equip_19'
)

cursor = conn.cursor()

cursor.execute("SHOW TABLES")
tablas = [t[0] for t in cursor.fetchall()]


dfs = {tabla: pd.read_sql(f"SELECT * FROM {tabla}", conn) for tabla in tablas}

df = dfs['Tourist_Accommodation03112025']

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   apartment_id                 8000 non-null   int64  
 1   name                         7997 non-null   object 
 2   description                  7946 non-null   object 
 3   host_id                      8000 non-null   int64  
 4   neighbourhood_name           8000 non-null   object 
 5   neighbourhood_district       4861 non-null   object 
 6   room_type                    8000 non-null   object 
 7   accommodates                 8000 non-null   int64  
 8   bathrooms                    7957 non-null   object 
 9   bedrooms                     7961 non-null   object 
 10  beds                         7992 non-null   float64
 11  amenities_list               7983 non-null   object 
 12  price                        7829 non-null   float64
 13  minimum_nights    

# Tratamiento de nulos: Precio

In [50]:
df.isnull().sum()

apartment_id                      0
name                              3
description                      54
host_id                           0
neighbourhood_name                0
neighbourhood_district         3139
room_type                         0
accommodates                      0
bathrooms                        43
bedrooms                         39
beds                              8
amenities_list                   17
price                           171
minimum_nights                    0
maximum_nights                    0
has_availability                550
availability_30                   0
availability_60                   0
availability_90                   0
availability_365                  0
number_of_reviews                 0
first_review_date              1614
last_review_date               1615
review_scores_rating           1696
review_scores_accuracy         1705
review_scores_cleanliness      1699
review_scores_checkin          1710
review_scores_communication 

In [52]:
precio_invalido = df['price'].isnull() | (df['price'] == 0)

df_precios_invalidos = df.loc[precio_invalido].copy()

# Comprobar los precios_inválidos
print(f"✅ Total de registros con precio inválido (Nulos O Cero): {len(df_precios_invalidos)}")
print(f"   Filas con precio nulo (NaN): {df_precios_invalidos['price'].isnull().sum()}")
print(f"   Filas con precio a cero (0): {(df_precios_invalidos['price'] == 0).sum()}")


✅ Total de registros con precio inválido (Nulos O Cero): 171
   Filas con precio nulo (NaN): 171
   Filas con precio a cero (0): 0


# Conversión de Tipos

CONVERSIÓN DE FECHAS

In [56]:
df["first_review_date"] = pd.to_datetime(df["first_review_date"], errors="coerce")
df["last_review_date"] = pd.to_datetime(df["last_review_date"], errors="coerce")
df["insert_date"] = pd.to_datetime(df["insert_date"], errors="coerce")

CONVERSIÓN A BOOLEANO

In [59]:
df['has_availability'] = df['has_availability'].replace({'VERDADERO': True})
df['has_availability'] = df['has_availability'].astype(bool)

df['is_instant_bookable'] = df['is_instant_bookable'].replace({'VERDADERO': True, 'FALSO': False})
df['is_instant_bookable'] = df['is_instant_bookable'].astype(bool)

CONVERSIÓN A INTEGRO

In [62]:
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce').astype('Int64')
df["bedrooms"] = pd.to_numeric(df["bedrooms"], errors='coerce').astype('Int64')
df['beds'] = pd.to_numeric(df['beds'], errors='coerce').astype('Int64')
df['is_instant_bookable'] = df['is_instant_bookable'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   apartment_id                 8000 non-null   int64         
 1   name                         7997 non-null   object        
 2   description                  7946 non-null   object        
 3   host_id                      8000 non-null   int64         
 4   neighbourhood_name           8000 non-null   object        
 5   neighbourhood_district       4861 non-null   object        
 6   room_type                    8000 non-null   object        
 7   accommodates                 8000 non-null   int64         
 8   bathrooms                    7957 non-null   Int64         
 9   bedrooms                     7961 non-null   Int64         
 10  beds                         7992 non-null   Int64         
 11  amenities_list               7983 non-null 

# Duplicados

COMPROBACIÓN DUPLICADOS APARTMENT_ID

In [66]:
df['apartment_id'].duplicated().any()

True

In [68]:
total_duplicados = df['apartment_id'].duplicated().sum()
print(f"Total de registros duplicados encontrados en 'apartment_id': {total_duplicados}")

Total de registros duplicados encontrados en 'apartment_id': 307


ELIMINACIÓN DE DUPLICADOS DE APARTMENT_ID, MANTENIENDO LOS MÁS RECIENTES SEGÚN INSERT_DATE

In [71]:
df[df['apartment_id'].duplicated(keep=False)]

df = df.sort_values(['apartment_id', 'insert_date'])

df = df.drop_duplicates(subset='apartment_id', keep='last')

In [73]:
df['apartment_id'].duplicated().any()

False

In [75]:
df.shape

(7693, 35)

# Estandarización de categorías y texto

NEIGHBOURHOOD

In [79]:
def clean_neighbourhood(text):
    if pd.isna(text) or text == '':
        return pd.NA
    text = str(text)
    #text = text.replace("�", "")
    text = re.sub(r"[^\p{L}\p{N}\s'\-]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = text.title()
    
    return text if text else pd.NA

In [81]:
failed_text_name = []
for i in df['neighbourhood_name']:
    if i.__contains__("�"):
        failed_text_name.append(i)

In [83]:
name_mapping = {
    'C�rmenes': 'Cármenes',
    'Torroella de Fluvi�': 'Torroella de Fluvià',
    "el Camp d'en Grassot i Gr�cia Nova": "el Camp d'en Grassot i Gràcia Nova",
    'Alc�dia': 'Alcúdia',
    "Castell� d'Emp�ries": "Castelló d'Empúries",
    'Ciudad Jard�n': 'Ciudad Jardín',
    'la Vila de Gr�cia': 'la Vila de Gràcia',
    'Ni�o Jes�s': 'Niño Jesús',
    'la Sagrada Fam�lia': 'la Sagrada Família',
    'el Putxet i el Farr�': 'el Putxet i el Farró',
    'el Barri G�tic': 'el Barri Gòtic',
    'S�ller': 'Sóller',
    'Santany�': 'Santanyí',
    'la Vila Ol�mpica del Poblenou': 'la Vila Olímpica del Poblenou',
    'San Ferm�n': 'San Fermín',
    'Llub�': 'Llubí',
    'San Juli�n': 'San Julián',
    'San Bartolom�': 'San Bartolomé',
    'Calvi�': 'Calvià',
    'Pollen�a': 'Pollença',
    'Encarnaci�n, Regina': 'Encarnación, Regina',
    'Cadaqu�s': 'Cadaqués',
    'Mah�n': 'Mahón',
    'Sarri�': 'Sarrià',
    'Palam�s': 'Palamós',
    'Celr�': 'Celrà',
    'Torroella de Montgr�': 'Torroella de Montgrí',
    'Can Bar�': 'Can Baró',
    'Arg�elles': 'Argüelles',
    'Llan��': 'Llançà',
    'Pac�fico': 'Pacífico',
    'Proven�als del Poblenou': 'Provençals del Poblenou',
    'Marratx�': 'Marratxí',
    'Sant Feliu de Gu�xols': 'Sant Feliu de Guíxols',
    'Diagonal Mar i el Front Mar�tim del Poblenou': 'Diagonal Mar i el Front Marítim del Poblenou',
    'Santa Mar�a del Cam�': 'Santa María del Camí',
    'Pont�s': 'Pontós',
    'Le�n XIII, Los Naranjos': 'León XIII, Los Naranjos',
    'Sant Lloren� des Cardassar': 'Sant Llorenç des Cardassar',
    'Sant Llu�s': 'Sant Lluís',
    'Sant Gen�s dels Agudells': 'Sant Genís dels Agudells',
    'Art�': 'Artà',
    'el Baix Guinard�': 'el Baix Guinardó',
    'Tortell�': 'Tortellà',
    'Alar�': 'Alaró',
    'Hispanoam�rica': 'Hispanoamérica',
    'Cornell� del Terri': 'Cornellà del Terri',
    'B�ger': 'Búger',
    'Llad�': 'Lladó',
    'Palau de Santa Eul�lia': 'Palau de Santa Eulàlia',
    'el Congr�s i els Indians': 'el Congrés i els Indians',
    'el Bes�s i el Maresme': 'el Besòs i el Maresme',
    'Dey�': 'Deià',
    'Casco Hist�rico de Vallecas': 'Casco Histórico de Vallecas',
    'Carretera de Carmona, Mar�a Auxiliadora, Fontanal': 'Carretera de Carmona, María Auxiliadora, Fontanal',
    'Ur�s': 'Urús',
    'Sant Mart� Vell': 'Sant Martí Vell',
    'Vila�r': 'Vilafran',
    'Vidr�': 'Vidré',
    'Sant Mart� de Proven�als': 'Sant Martí de Provençals',
    'el Guinard�': 'el Guinardó',
    'Concepci�n': 'Concepción',
    'Bescan�': 'Bescanó',
    'Jer�nimos': 'Jerónimos',
    'Moscard�': 'Moscardó',
    'Doctor Barraquer, G. Renfe, Policl�nico': 'Doctor Barraquer, G. Renfe, Policlínico',
    'Seriny�': 'Serinyà',
    "Cru�lles, Monells i Sant Sadurn� de l'Heura": "Cruïlles, Monells i Sant Sadurní de l'Heura",
    'Montu�ri': 'Montuïri',
    'el Tur� de la Peira': 'el Turó de la Peira',
    'Ventall�': 'Ventalló',
    'Fontarr�n': 'Fontarrón',
    'San Andr�s': 'San Andrés',
    'El Roc�o': 'El Rocío',
    "Bellcaire d'Empord�": "Bellcaire d'Empordà",
    'Nervi�n': 'Nervión',
    'Santa Eug�nia': 'Santa Eugènia',
    'Besal�': 'Besalú',
    'Nueva Espa�a': 'Nueva España',
    'Ull�': 'Ullà',
    'Pe�agrande': 'Peñagrande',
    'Casco Hist�rico de Barajas': 'Casco Histórico de Barajas',
    'San Jos� Obrero': 'San José Obrero',
    'La Palmilla, Doctor Mara��n': 'La Palmilla, Doctor Marañón',
    'Barrio Le�n': 'Barrio León',
    'Puigcerd�': 'Puigcerdà',
    'El Tard�n, El Carmen': 'El Tardón, El Carmen',
    'Hell�n': 'Hellín',
    'Opa�el': 'Opañel',
    'Sant Mart� de Ll�mena': 'Sant Martí de Llémena',
    'Foix�': 'Foixà',
    'Casco Hist�rico de Vic�lvaro': 'Casco Histórico de Vicálvaro',
    "La Tallada d'Empord�": "La Tallada d'Empordà",
    'Vallfogona de Ripoll�s': 'Vallfogona de Ripollès',
    'Campdev�nol': 'Campdevànol',
    'Rab�s': 'Rabós',
    'Regenc�s': 'Regencós',
    'Zof�o': 'Zofío',
    'Sant Pau de Seg�ries': 'Sant Pau de Segúries',
    'Torre Bar�': 'Torre Baró',
    'Tiro de L�nea, Santa Genoveva': 'Tiro de Línea, Santa Genoveva',
    'Tim�n': 'Timón',
    'Ma�anet de la Selva': 'Maçanet de la Selva',
    'Entrev�as': 'Entrevías',
    'Parlav�': 'Parlavà',
    'Sant Juli� de Ramis': 'Sant Julià de Ramis',
    'Mai� de Montcal': 'Maçanet de Montcal',
    'B�scara': 'Bàscara',
    'Arb�cies': 'Arbúcies',
    'Vilaju�ga': 'Vilajuïga'
}

In [85]:
df["neighbourhood_name"] = df["neighbourhood_name"].replace(name_mapping)

In [87]:
df['neighbourhood_district'] = df['neighbourhood_district'].fillna("")
df['neighbourhood_district']

0                    
1              Latina
2       Casco Antiguo
3            Eixample
4                    
            ...      
7995         Eixample
7996    Casco Antiguo
7997                 
7998       Sant Mart�
7999                 
Name: neighbourhood_district, Length: 7693, dtype: object

In [89]:
failed_text_district=[]
for i in df['neighbourhood_district']:
    if i.__contains__("�"):
        failed_text_district.append(i)

In [91]:
name_mapping_district = {
    'Sant Mart�': 'Sant Martí',
    'Gr�cia': 'Gràcia',
    'Sarri�-Sant Gervasi': 'Sarrià-Sant Gervasi',
    'Sants-Montju�c': 'Sants-Montjuïc',
    'Horta-Guinard�': 'Horta-Guinardó',
    'Chamart�n': 'Chamartín',
    'Tetu�n': 'Tetuán',
    'Nervi�n': 'Nervión',
    'Chamber�': 'Chamberí',
    'Vic�lvaro': 'Vicálvaro'
}

In [93]:
df["neighbourhood_district"] = df["neighbourhood_district"].replace(name_mapping_district)

In [95]:
df['neighbourhood_name'] = df['neighbourhood_name'].apply(clean_neighbourhood)
df['neighbourhood_district'] = df['neighbourhood_district'].apply(clean_neighbourhood)

In [97]:
df[['neighbourhood_name', 'neighbourhood_district']].head(10)

,neighbourhood_name,neighbourhood_district
0,Centro,<NA>
1,Cármenes,Latina
2,San Vicente,Casco Antiguo
3,L'Antiga Esquerra De L'Eixample,Eixample
4,Quart,<NA>
5,Torroella De Fluvià,<NA>
6,El Camp De L'Arpa Del Clot,Sant Martí
7,La Dreta De L'Eixample,Eixample
8,Embajadores,Centro
9,El Camp D'En Grassot I Gràcia Nova,Gràcia


In [99]:
df["neighbourhood_name"]

0                                Centro
1                              Cármenes
2                           San Vicente
3       L'Antiga Esquerra De L'Eixample
4                                 Quart
                     ...               
7995                        Sant Antoni
7996                             Arenal
7997                           Felanitx
7998            Provençals Del Poblenou
7999                              Roses
Name: neighbourhood_name, Length: 7693, dtype: object

In [101]:
df["neighbourhood_district"]

0                <NA>
1              Latina
2       Casco Antiguo
3            Eixample
4                <NA>
            ...      
7995         Eixample
7996    Casco Antiguo
7997             <NA>
7998       Sant Martí
7999             <NA>
Name: neighbourhood_district, Length: 7693, dtype: object

ESTANDARIZAR CITY

In [104]:
df['city'].value_counts()

barcelona    2261
madrid       1645
mallorca     1265
girona       1198
sevilla       403
malaga        394
valencia      360
menorca       167
Name: city, dtype: int64

In [106]:
city_mapping = {
    'Seville': 'Sevilla',
    'Madrid': 'Madrid',
    'Barcelona': 'Barcelona',
    'Valencia': 'Valencia',
    'Malaga': 'Málaga',
    'Málaga': 'Málaga',        
    'València': 'Valencia',    
    'Sevilla': 'Sevilla',
    'barcelona': 'Barcelona',
    'MADRID': 'Madrid',
    'valencia': 'Valencia',
    'malaga': 'Málaga',
    'sevilla': 'Sevilla',
    'girona': 'Girona',
    'Girona': 'Girona',
    'mallorca': 'Mallorca',
    'Mallorca': 'Mallorca',
    'Menorca': 'Menorca',
    'menorca': 'Menorca',
    'madrid': 'Madrid',
    'barcelona': 'Barcelona',
    'valencia': 'Valencia',
    'malaga': 'Málaga',
    'sevilla': 'Sevilla',
    'girona': 'Girona',
    'mallorca': 'Mallorca',
    'menorca': 'Menorca',
}

In [108]:
valid_cities = {
    'Barcelona',
    'Madrid',
    'Sevilla',
    'Valencia',
    'Málaga',
    'Girona',
    'Mallorca',
    'Menorca'
}

In [110]:
df['city_clean'] = df['city'].astype(str).str.strip()

# Crea una versión en minúsculas del mapeo para la coincidencia que no distinga entre mayúsculas y minúsculas.
mapping_lower = {k.lower(): v for k, v in city_mapping.items()}

# Normalizar
df['city_normalized'] = df['city_clean'].str.lower().map(mapping_lower)

# Filtrar solo cidades válidas
df = df[df['city_normalized'].isin(valid_cities)]

# Substituir colmuna original
df['city'] = df['city_normalized']
df = df.drop(columns=['city_clean', 'city_normalized'])

In [112]:
df[['city']].head(5)

,city
0,Málaga
1,Madrid
2,Sevilla
3,Barcelona
4,Girona


ESTANDARIZAR COUNTRY

In [115]:
df['country'] = 'Spain'
df[['country']].head(5)

,country
0,Spain
1,Spain
2,Spain
3,Spain
4,Spain


# Tratamiento de valores nulos

PRICE, CITY Y ROOM_TYPE

In [119]:
# Marcar inválidos
df['is_valid'] = True
df.loc[df['price'].isna() | df['city'].isna() | df['room_type'].isna(), 'is_valid'] = False
df_valid = df[df['is_valid']].copy()

In [121]:
df = df_valid

In [123]:
df.shape

(7534, 36)

Nulos en variables reviews


In [126]:
# Forzar nulos en reviews si no hay reviews
review_cols = [
    'first_review_date', 'last_review_date',
    'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location',
    'review_scores_value', 'reviews_per_month'
]
mask_no_reviews = df_valid['number_of_reviews'] == 0
df_valid.loc[mask_no_reviews, review_cols] = np.nan

# Imputar reviews_per_month
def calculate_reviews_per_month(row):
    if row['number_of_reviews'] > 0:
        if not pd.isna(row['first_review_date']):
            today = datetime.now()
            days_since_first = (today - row['first_review_date']).days
            if days_since_first > 0:
                return row['number_of_reviews'] / (days_since_first / 30)
        elif not pd.isna(row['last_review_date']):
            today = datetime.now()
            days_since_last = (today - row['last_review_date']).days
            if days_since_last > 0:
                return row['number_of_reviews'] / (days_since_last / 30)
    return np.nan

df_valid['reviews_per_month'] = df_valid.apply(calculate_reviews_per_month, axis=1)

# Amenities_list
df_valid['amenities_list'] = df_valid['amenities_list'].apply(lambda x: [] if pd.isna(x) or (isinstance(x, str) and x.strip() == '') else x)

# Validar coherencia
review_cols_no_nulls = df_valid.loc[(df_valid['number_of_reviews'] == 0) & (df_valid[review_cols].notna().any(axis=1))]
print(f"Registros inconsistentes: {len(review_cols_no_nulls)}")

Registros inconsistentes: 0


# Coherencia entre variables

COMPROBAR QUE MIN NIGHTS < MAX NIGHTS

In [130]:
df_error = df[df['minimum_nights'] > df['maximum_nights']]
df_error
# Sin resultados, así que todo está correcto

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid


COMPROBAR AVAILABILITY

In [133]:
df_hav = df[
    (df['availability_30'] > df['availability_60']) &
    (df['availability_60'] > df['availability_90']) &
    (df['availability_90'] > df['availability_365'])
].copy()

df_hav

# Sin resultados, así que todo está correcto

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid


In [135]:
cols_disp = ['availability_30', 'availability_60', 'availability_90', 'availability_365']

condicion_invalida = (
    (df['has_availability'] == True) & 
    (df[cols_disp].isna().all(axis=1))
)

df_invalidas = df[condicion_invalida]
print(df_invalidas)

# Empty Dataframe, así que todo está correcto

Empty DataFrame
Columns: [apartment_id, name, description, host_id, neighbourhood_name, neighbourhood_district, room_type, accommodates, bathrooms, bedrooms, beds, amenities_list, price, minimum_nights, maximum_nights, has_availability, availability_30, availability_60, availability_90, availability_365, number_of_reviews, first_review_date, last_review_date, review_scores_rating, review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value, is_instant_bookable, reviews_per_month, country, city, insert_date, is_valid]
Index: []

[0 rows x 36 columns]


LIMPIEZA AVAILABILITY

In [138]:
# Comprobamos coherencia entre los datos de las columnas has_availability y las cols_disp:
df_disponibilidad = df.loc[df['has_availability'] != True, 
                     ['has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365']]

df_disponibilidad
# No hay coherencia en estos casos

,has_availability,availability_30,availability_60,availability_90,availability_365
7,False,28,55,85,360
12,False,7,23,53,328
29,False,13,43,72,347
32,False,5,11,37,234
36,False,8,14,25,224
...,...,...,...,...,...
4762,False,16,46,76,76
4771,False,6,15,31,114
4791,False,13,43,73,163
4802,False,3,33,63,338


In [140]:
# Se define la condición de disponibilidad:
sidisponible = (df[cols_disp] > 0).any(axis=1)

# Se corrige la columna 'has_availability' en función de las cols_disp:
df.loc[(df['has_availability'] == False) & sidisponible, 'has_availability'] = True

In [142]:
# Comprobamos:
df[['has_availability'] + cols_disp].head()

,has_availability,availability_30,availability_60,availability_90,availability_365
0,True,7,20,40,130
1,True,0,0,0,162
2,True,26,31,31,270
3,True,9,23,49,300
4,True,0,19,49,312


COHERENCIA FECHAS INSERT_DATE ANTES QUE FIRST_REVIEW_DATE

In [145]:
df_fechas = df[
    (df['insert_date'] > df['first_review_date']) 
].copy()

totaldf = len(df_fechas)
print("Hay", totaldf, "casos en los que la 'insert_date' es posterior a la 'first_review_date', \n por lo que deducimos que se trata de anuncios que han sido actualizados y conservan reseñas antiguas.")


Hay 5922 casos en los que la 'insert_date' es posterior a la 'first_review_date', 
 por lo que deducimos que se trata de anuncios que han sido actualizados y conservan reseñas antiguas.


COHERENCIA FECHAS FIRST_REVIEW DATE ANTES QUE LAST_REVIEW_DATE

In [148]:
df_fechas2 = df[
    (df['first_review_date'] > df['last_review_date'])
].copy()

totaldf2 = len(df_fechas2)

print("Hay", totaldf2, "casos en los que la 'first_review_date' es posterior " \
"a la 'last_review_date', \n por lo que puede que haya habido un error en la adquisición de los datos") 

Hay 265 casos en los que la 'first_review_date' es posterior a la 'last_review_date', 
 por lo que puede que haya habido un error en la adquisición de los datos


In [150]:
# Se intercambian los datos entre las columnas first_review_date' y 'last_review_date' en los casos de incoherencia para deshacer esta inconsistencia  
def arreglar_fechas_reviews(row):
    if row['first_review_date'] > row['last_review_date']:
        row['first_review_date'], row['last_review_date'] = row['last_review_date'], row['first_review_date']
    return row

df = df.apply(arreglar_fechas_reviews, axis=1)

# Cambio de la presentación del dato

CAMBIO DE SCORES RATING A 0 - 100 Y TODOS LOS DEMÁS A 0 - 10

In [154]:
df['review_scores_rating'] = df['review_scores_rating'] / 10
df['review_scores_accuracy'] = df['review_scores_accuracy'] / 10
df['review_scores_checkin'] = df['review_scores_checkin'] / 10
df['review_scores_cleanliness'] = df['review_scores_cleanliness'] / 10
df['review_scores_communication'] = df['review_scores_communication'] / 10
df['review_scores_location'] = df['review_scores_location'] / 10
df['review_scores_value'] = df['review_scores_value'] / 10


In [156]:
df['review_scores_value'].head(10)

0    10.0
1     9.0
2    10.0
3     9.0
4    10.0
5    10.0
6     9.0
7     9.0
8     9.0
9     9.0
Name: review_scores_value, dtype: float64

# Tratamiento de más valores Nulos

ELIMINACIÓN NULOS EN BEDROOMS

In [160]:
bedrooms_nulas = df.loc[df['bedrooms'].isnull(), ['bedrooms', 'room_type', 'accommodates']]
print("Hay", len(bedrooms_nulas), "casos nulos en 'bedrooms'.")


Hay 39 casos nulos en 'bedrooms'.


In [162]:
# 1. Imputación para 'Private room' (Hay una sola habitación)
df.loc[(df['room_type'] == 'Private room') & (df['bedrooms'].isnull()), 'bedrooms'] = 1

# 2. Imputación para 'Entire home/apt' (Se calcula en base a los acommodates, para poner una habitación para acda 2, redondeo hacia arriba)
mask_entire = (df['room_type'] == 'Entire home/apt') & (df['bedrooms'].isnull())

# Se imputa el número de habitaciones como la capacidad dividida por 2, redondeado hacia arriba
df.loc[mask_entire, 'bedrooms'] = (df.loc[mask_entire, 'accommodates'] / 2).apply(np.ceil)

In [164]:
# Comprobación
df[['bedrooms', 'room_type', 'accommodates']]

,bedrooms,room_type,accommodates
0,1,Private room,2
1,1,Private room,1
2,2,Entire home/apt,4
3,1,Private room,2
4,2,Private room,5
...,...,...,...
7995,1,Private room,1
7996,3,Entire home/apt,6
7997,1,Entire home/apt,2
7998,2,Private room,3


ELIMINACIÓN NULOS EN BATHROOMS

In [167]:
df.loc[df['bathrooms'].isnull(), ['bathrooms', 'room_type', 'accommodates']]
df.loc[df['bathrooms'].isnull(), 'bathrooms'] = 1

ELIMINACIÓN NULOS EN BEDS (se usa la mediana del grupo según las variables room_type, bedrooms y accomodates)

In [170]:
def rellenar_nulos_con_mediana_grupo(serie):
    """
    Calcula la mediana de la serie y la usa para rellenar solo los valores NaN de esa serie.
    """
    return serie.fillna(serie.median())

#    Agrupamos por 'room_type' Y 'bedrooms' antes de aplicar la transformación a 'camas'.
df['camas_imputadas'] = df.groupby(['room_type', 'bedrooms', 'accommodates'])['beds'].transform(rellenar_nulos_con_mediana_grupo)

# Sustitución y Limpieza de la columna temporal
df['beds'] = df['camas_imputadas']
df = df.drop(columns=['camas_imputadas'])

In [172]:
# Comprobamos
print(f"Valores nulos restantes en 'beds': {df['beds'].isnull().sum()}")

Valores nulos restantes en 'beds': 0


ELIMINACIÓN DE NULOS EN REVIEWS

In [175]:
df.loc[df['reviews_per_month'].isnull(), ['reviews_per_month', 'number_of_reviews']]

,reviews_per_month,number_of_reviews
61,NaN,0
103,NaN,0
137,NaN,0
142,NaN,0
168,NaN,0
...,...,...
7991,NaN,0
7992,NaN,0
7993,NaN,0
7995,NaN,0


In [177]:
# se les da valor 0
df.loc[
    (df['reviews_per_month'].isnull()) & (df['number_of_reviews'] == 0),
    'reviews_per_month'
] = 0

In [179]:
# Comprobamos:
df.loc[df['number_of_reviews'] == 0, 
       ['reviews_per_month', 
        'number_of_reviews', 
        'first_review_date', 
        'last_review_date', 
        'review_scores_rating', 
        'review_scores_accuracy', 
        'review_scores_checkin', 
        'review_scores_cleanliness', 
        'review_scores_communication', 
        'review_scores_location', 
        'review_scores_value']]

,reviews_per_month,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_value
61,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7991,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7992,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7993,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7995,0.0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
colsrev = [
    'reviews_per_month',
    'number_of_reviews',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_checkin',
    'review_scores_cleanliness',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]

df.loc[df['number_of_reviews'] == 0, colsrev] = 0

In [183]:
df.isnull().sum()

apartment_id                      0
name                              3
description                      49
host_id                           0
neighbourhood_name                0
neighbourhood_district         2927
room_type                         0
accommodates                      0
bathrooms                         0
bedrooms                          0
beds                              0
amenities_list                    0
price                             0
minimum_nights                    0
maximum_nights                    0
has_availability                  0
availability_30                   0
availability_60                   0
availability_90                   0
availability_365                  0
number_of_reviews                 0
first_review_date              1488
last_review_date               1489
review_scores_rating             83
review_scores_accuracy           92
review_scores_cleanliness        86
review_scores_checkin            97
review_scores_communication 

¿FIRST_REVIEW_DATE Y LAST_REVIEW_DATE?

In [186]:
# df.loc[
#     df['last_review_date'].isnull() & df['first_review_date'].notnull(),
#     ['first_review_date', 'last_review_date', 'number_of_reviews', 'reviews_per_month']
# ]

df.loc[
    df['first_review_date'].isnull() & df['last_review_date'].notnull(),
    ['first_review_date', 'last_review_date', 'number_of_reviews', 'reviews_per_month']
]

,first_review_date,last_review_date,number_of_reviews,reviews_per_month
6393,NaT,2018-03-05,1,0.010707


COMPROBACIÓN NULOS DE NEIGHBOURHOOD DISTRICT

In [189]:
df.loc[df["neighbourhood_district"].isna(), "neighbourhood_name"].value_counts()

df_nulls = df[df["neighbourhood_district"].isna()][["neighbourhood_name", "city", "apartment_id"]]

df_nulls

,neighbourhood_name,city,apartment_id
0,Centro,Málaga,11964
4,Quart,Girona,35801
5,Torroella De Fluvià,Girona,48764
27,Palma De Mallorca,Mallorca,164621
30,Alcúdia,Mallorca,178456
...,...,...,...
7989,Castelló D'Empúries,Girona,32361345
7992,Llucmajor,Mallorca,32379297
7994,Santanyí,Mallorca,32391703
7997,Felanitx,Mallorca,32395123


In [191]:
df[["neighbourhood_district", "neighbourhood_name"]]

,neighbourhood_district,neighbourhood_name
0,<NA>,Centro
1,Latina,Cármenes
2,Casco Antiguo,San Vicente
3,Eixample,L'Antiga Esquerra De L'Eixample
4,<NA>,Quart
...,...,...
7995,Eixample,Sant Antoni
7996,Casco Antiguo,Arenal
7997,<NA>,Felanitx
7998,Sant Martí,Provençals Del Poblenou


# Nuevas columnas

CREACIÓN DE NUEVA COLUMNA

In [195]:
df['ocupacion_mes'] = 30 - df['availability_30']

# EDA

COMPROBACIÓN OUTLIERS

In [199]:
pd.set_option('display.max_columns', None)
df[df['price'] > 6000]

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,beds,amenities_list,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid,ocupacion_mes
2390,12970898,APARTAMENTO CENTRICO-EL CARMEN,Apartamento totalmente reformado en 2018. Dos ...,71363084,La Xerea,Ciutat Vella,Entire home/apt,6,2,3,7.0,"Paid parking on premises, Shampoo, Oven, Host ...",6071.0,2,60,True,0,0,3,3,38,2018-09-03,2020-08-23,93.0,10.0,9.0,10.0,10.0,10.0,9.0,0,0.435115,Spain,Valencia,2020-09-28,True,30


In [201]:
pd.set_option('display.max_columns', None)
df[df['beds'] > 25]

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,beds,amenities_list,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_valid,ocupacion_mes


In [203]:
df.to_csv('tourist_accommodation_03112025_clean.csv', index=False)  

In [205]:
df.isnull().sum()

apartment_id                      0
name                              3
description                      49
host_id                           0
neighbourhood_name                0
neighbourhood_district         2927
room_type                         0
accommodates                      0
bathrooms                         0
bedrooms                          0
beds                              0
amenities_list                    0
price                             0
minimum_nights                    0
maximum_nights                    0
has_availability                  0
availability_30                   0
availability_60                   0
availability_90                   0
availability_365                  0
number_of_reviews                 0
first_review_date              1488
last_review_date               1489
review_scores_rating             83
review_scores_accuracy           92
review_scores_cleanliness        86
review_scores_checkin            97
review_scores_communication 